In [1]:
import os
import re
import csv
import json
import time
import base64
import chromadb
import openai
import pandas as pd
import streamlit as st

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_qa_with_sources_chain
from langchain.chains import ConversationalRetrievalChain

from langchain_community.document_loaders.csv_loader import CSVLoader



with open('config.json') as file:
    config = json.load(file)

openai.api_key = config.get('key')
os.environ['OPENAI_API_KEY'] = config["key"]



loader = CSVLoader("messages.csv",encoding="utf-8", source_column="text")
data = loader.load()

text_splitter =RecursiveCharacterTextSplitter(

    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len,
    is_separator_regex=False

)
documents = text_splitter.split_documents(data)

embeddings_model = OpenAIEmbeddings()
db = Chroma.from_documents(documents, embeddings_model)

retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5})
docs = retriever.get_relevant_documents("question to test")
llm_src = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
docs

c:\Users\savalgis\AppData\Local\miniconda3\envs\Hackathon_2024\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\savalgis\AppData\Local\miniconda3\envs\Hackathon_2024\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import Ch

[Document(page_content='question might answer things.', metadata={'row': 499, 'source': "We understand and that striving is what causes real learning. I'm just saying its not efficient to spend hours trying to debug something when a short question might answer things."}),
 Document(page_content='my thought and questions out there?', metadata={'row': 108, 'source': 'is there a general rule on how to start a topic here or do i just throw my thought and questions out there?'}),
 Document(page_content='sandbox just for testing', metadata={'row': 728, 'source': "Well, I don't have a domain to associate it with, I'm trying to use their sandbox just for testing"}),
 Document(page_content='=> {}), how can I do this test?', metadata={'row': 61, 'source': '@pmiralopes Best might be to authenticate the user in the test. Otherwise you can consider making your test call like service.remove(... , { _isTest: true } } and conditioning your auth hook like iff(context => !context.params._isTest, authhoo